In [27]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("241212_01_MLlib_Clustering").getOrCreate()

## DATA LOAD

In [28]:
# 데이터 생성
data = [
    (0, 0, 4.0),  # user 0 rated item 0 with 4.0
    (0, 1, 2.0),
    (1, 1, 3.0),
    (1, 2, 1.0),
    (2, 0, 5.0),
    (2, 2, 4.0)
]

columns = ["user_id","item_id","rating"]

In [29]:
rating_df = spark.createDataFrame(data,columns)
rating_df.show()

+-------+-------+------+
|user_id|item_id|rating|
+-------+-------+------+
|      0|      0|   4.0|
|      0|      1|   2.0|
|      1|      1|   3.0|
|      1|      2|   1.0|
|      2|      0|   5.0|
|      2|      2|   4.0|
+-------+-------+------+



## 전처리

In [30]:
# user, item - rating 정보를 >> 사용자 그룹을 만든다. > 모델 > 예측 결과 - 그룹

In [31]:
# 피치 선택, 전처리 > 모델 학습 > 예측

In [32]:
user_item_matrix = rating_df.groupBy("user_id").pivot("item_id").avg("rating").fillna(0)
user_item_matrix.show()

+-------+---+---+---+
|user_id|  0|  1|  2|
+-------+---+---+---+
|      0|4.0|2.0|0.0|
|      1|0.0|3.0|1.0|
|      2|5.0|0.0|4.0|
+-------+---+---+---+



## feature vector

In [33]:
from pyspark.ml.feature import VectorAssembler

In [34]:
assembler = VectorAssembler(inputCols=["0","1","2"], outputCol='features')

In [35]:
user_features = assembler.transform(user_item_matrix)
user_features.show()

+-------+---+---+---+-------------+
|user_id|  0|  1|  2|     features|
+-------+---+---+---+-------------+
|      0|4.0|2.0|0.0|[4.0,2.0,0.0]|
|      1|0.0|3.0|1.0|[0.0,3.0,1.0]|
|      2|5.0|0.0|4.0|[5.0,0.0,4.0]|
+-------+---+---+---+-------------+



## 모델 생성 > 학습

In [36]:
from pyspark.ml.clustering import KMeans

#모델 생성
kmeans = KMeans(k=2, seed=1, featuresCol="features", predictionCol="cluster")
kmeans

KMeans_25ff45222aea

In [37]:
#모델 학습
model = kmeans.fit(user_features)
model

KMeansModel: uid=KMeans_25ff45222aea, k=2, distanceMeasure=euclidean, numFeatures=3

In [38]:
#모델을 이용한 예측
clusters = model.transform(user_features)

In [39]:
# 유저의 클러스터링 결과
clusters.show()

+-------+---+---+---+-------------+-------+
|user_id|  0|  1|  2|     features|cluster|
+-------+---+---+---+-------------+-------+
|      0|4.0|2.0|0.0|[4.0,2.0,0.0]|      0|
|      1|0.0|3.0|1.0|[0.0,3.0,1.0]|      0|
|      2|5.0|0.0|4.0|[5.0,0.0,4.0]|      1|
+-------+---+---+---+-------------+-------+



In [ ]:
## 사용자 그룹화 : 유사한 취향의 사용자끼리 그룹으로 묶어주는 것
## 아이템 그룹화 : 아이템 간의 군집화를 통해 사용자에게 추천해 줄 수 있음.